In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl

import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)

%matplotlib inline
%load_ext autoreload                                                                                                                                                                                                
%autoreload 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
if not os.path.isdir('DampedNewtonlogexpstabilizationtests_images'):
    os.makedirs('DampedNewtonlogexpstabilizationtests_images')

In [ ]:
"""To compute distance matrix"""
def distmat(x,y):
    return np.sum( x**2,0 )[:,None] + np.sum( y**2,0 )[None,:] - 2*x.transpose().dot(y)

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP(u,K,v):
    u = u.reshape(u.shape[0],)
    v = v.reshape(v.shape[0],)
    return u[:,None]*K*v[None,:]

def plotp(x, col,plt, scale=200, edgecolors="k"):
  return plt.scatter( x[0,:], x[1,:], s=scale, edgecolors=edgecolors,  c=col, cmap='plasma', linewidths=2 )

In [ ]:
def generate_data(N):
    """
     N is a list of the size of the data on x and y
    """
    x = np.random.rand( 2,N[0] )-0.5
    theta = 2*np.pi*np.random.rand( 1,N[1] )
    r = 0.8+.2*np.random.rand( 1,N[1] )
    y = np.vstack( ( r*np.cos( theta ),r*np.sin( theta ) ) )
    return x,y


In [ ]:
N = [ 1000,1500 ]
x,y = generate_data(N)

In [ ]:
import computational_OT

In [ ]:
epsilons = [ 1.0, 0.5, 0.3, 0.1, 0.09, 0.05, 0.03,0.02,0.01, 0.001 ]


## Sinkhorn

In [ ]:
# Sinkhorn
print("Sinkhorn.... ")
SinkhornP = []
results_Sinkhorn = []
times_Sinkhorn = []
Pmatrix_dist_linVSsinkhorn = []
for eps in epsilons:

  
  #Cost matrix
  C = distmat(x,y)
  
  # a and b
  a = normalize(np.ones(N[0]))
  a = a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[1]))
  b = b.reshape(b.shape[0],-1)



  #Kernel
  K = np.exp(-C/eps)


  print( "Doing for (",N[0],N[1],")." )
  print( " |- Iterating" )

  #Inflating
  u = a
  v = b

  start = time.time()
  Optimizer = computational_OT.Sinkhorn( K,a,b,u,v,eps)
  out = Optimizer._update(maxiter=10000)
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append(1e-3*(end-start) )
  print( " |- Computing P" )
  print( "" )
  SinkhornP.append( GetP( out['u'],K,out['v'] ) )
  

In [ ]:
plt.figure( figsize = (20,7) )

plt.subplot(2,1,1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn) ):
  error=np.asarray( results_Sinkhorn[i]['error_a'] )+np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error,label = 'Sinkhorn for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )
plt.yscale( 'log' )
plt.legend()
plt.xlabel("Iterations")
plt.ylabel("Error in log-scale")

plt.savefig("DampedNewtonlogexpstabilizationtests_images/ConvergenceSinkhornvaryingepsilon.png")
plt.show()

## Damped Newton with log-exp stabilization

In [ ]:
rho = 0.95
c = 0.05
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []
Hessians_DampedNewton = []

#epsilons=[0.05,0.08,0.1]
# epsilons=[0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1.0 ]
#epsilons=[0.3]
for eps in epsilons:
    
    # Line Search
    print( "Damped Newton for epsilon="+str(eps)+":" )    
    #Cost matrix
    C = distmat(x,y)

    # a and b
    a = normalize( np.ones( N[0] ) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones( N[1] ) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)
    f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewtonLogexp( K,a,b,f,g,eps,rho,c )
    out = Optimizer._update( stabilization_noise = 1e-4, maxiter = 50 )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append( 1e-3*(end-start) )
    print( " |- Computing P" )
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    print( " |- Recording (unstabilized) Hessian \n" )

    mat  = -eps*Optimizer.Hessian
    diag = 1/np.sqrt( np.vstack( (a,b) ) ).flatten()
    mat = diag[:,None]*mat*diag[None,:]
    Hessians_DampedNewton.append( mat )


In [ ]:
plt.figure( figsize = (20,7) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewton)):
  error = np.asarray( results_DampedNewton[i]['error_a'] )+np.asarray( results_DampedNewton[i]['error_b'] )
  plt.plot( error,label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "DampedNewtonlogexpstabilizationtests_images/ErrorDampedNewton.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

In [ ]:
plt.figure( figsize = (20,7) )
plt.title( "$$" )

plt.title( "Objective Function" )
for i in range(len(results_DampedNewton)):
  plt.plot( np.asarray( results_DampedNewton[i]["objectives"] ),label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.legend()
plt.savefig("DampedNewtonlogexpstabilizationtests_images/ObjectiveDampedNewton.png")
plt.show()


In [ ]:
plt.figure( figsize = (20,7) )
plt.subplot(2,1,1),
plt.title( "Alpha" )

for i in range(len(results_DampedNewton)):
  plt.plot( np.asarray( results_DampedNewton[i]['linesearch_steps'] ),label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend()
# plt.yscale( 'log')
plt.savefig("DampedNewtonlogexpstabilizationtests_images/AlphaDampedNewton.png")
plt.show()

### Time plot of Sinkhorn vs Damped Newton

In [ ]:
plt.figure( figsize = (20,7) )

plt.subplot(2,1,1),
plt.title( "Time Plot" )
plt.plot( epsilons,times_Sinkhorn, linewidth = 2,marker = 'o' )
plt.plot( epsilons,times_DampedNewton, linewidth = 2,marker = 'o'  )


plt.legend(['Sinkhorn Time','Damped Newton Time'])
plt.xlabel("$\epsilon$")
plt.ylabel("Time in ms")

plt.savefig("DampedNewtonlogexpstabilizationtests_images/Timeplot_SinkhornvsDampedNewton.png")
plt.show()

In [ ]:
def spectral_decomposition(mat):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort(eig)
    eig = eig[sorting_indices]
    v   = v[:, sorting_indices]
    
    print( "List of smallest eigenvalues: ", eig[:10])
    print( "List of largest  eigenvalues: ", eig[-10:])

    return eig,v

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon="+str(eps) )
    ev = spectral_decomposition( Hessians_DampedNewton[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")


In [ ]:
fig,ax = plt.subplots( figsize=(30,3),nrows=1, ncols=len(epsilons), sharey=True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
plt.subplots_adjust( wspace=0,hspace=0 )
plt.savefig("DampedNewtonlogexpstabilizationtests_images/eigenhistunstabilized.png")
plt.show()